# Scraping script for obtaining data on data related jobs in Malaysia

In [49]:
import time
from selenium import webdriver
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import requests
import re
import os
from lxml import etree
import lxml.html, lxml.html.clean
from urllib.parse import urlparse
from urllib.parse import parse_qs
import pandas as pd

#driver = webdriver.Chrome(ChromeDriverManager().install())
verbose_flag = True
# htmlSource = driver.page_source

In [8]:
data_engineer_site = r'https://www.jobstreet.com.my/en/job-search/data-engineer-jobs/'
main_site=r"https://www.jobstreet.com.my"

page = requests.get(data_engineer_site)

soup = BeautifulSoup(page.content, 'html.parser')



In [9]:
#find number of result pages
results = soup.find('select', attrs={"id":"pagination"})
options = results.find_all('option')

num_result_pages = int(options[-1]['value'])

In [10]:
jobslist = soup.find('div', attrs={'id':"jobList"}).find('div', attrs={'data-automation':'jobListing'})

In [11]:
jobs = jobslist.find_all('a', attrs={'class':'_1hr6tkx5 _1hr6tkx8 _1hr6tkxb sx2jih0 sx2jihf zcydq8h', 'href': re.compile(r'^\/en\/job\/')})

In [12]:
joblink = jobs[29].get('href')

In [13]:
job_fulllink = main_site + joblink
#job_fulllink = r"https://www.jobstreet.com.my/en/job/data-engineer-4834504?jobId=jobstreet-my-job-4834504&sectionRank=1&token=0~9d2b2714-12eb-4e27-b40c-24f3babd0c52&fr=SRP%20View%20In%20New%20Tab"
job_page = requests.get(job_fulllink)

job_soup = BeautifulSoup(job_page.content, 'html.parser')


In [14]:
parsed_url = urlparse(job_fulllink)
captured_value = parse_qs(parsed_url.query)
print(captured_value['jobId'])

['jobstreet-my-job-4846899']


In [15]:
jobs_dom = etree.HTML(str(job_soup))

In [16]:
job_title = jobs_dom.xpath('//*[@id="contentContainer"]/div/div[1]/div[1]/div[1]/div/div/div[1]/div/div/div[2]/div/div/div/div[1]/h1')[0].text
company_name = jobs_dom.xpath('//*[@id="contentContainer"]/div/div[1]/div[1]/div[1]/div/div/div[1]/div/div/div[2]/div/div/div/div[2]/span')[0].text
salary_range = jobs_dom.xpath('//*[@id="contentContainer"]/div/div[1]/div[1]/div[1]/div/div/div[2]/div/div/div/div[2]/span')[0].text

cleaner = lxml.html.clean.Cleaner(allow_tags=[''])
cleaned = cleaner.clean_html(etree.tostring(jobs_dom.xpath('//*[@id="contentContainer"]/div/div[2]/div/div[1]/div/div[1]/div/div[2]/div/span/div')[0]))
print()
job_description = cleaned.decode("utf-8").replace('<div>', '').replace('</div>','')
#job_description = etree.tostring(jobs_dom.xpath('//*[@id="contentContainer"]/div/div[2]/div/div[1]/div/div[2]/div/div[2]/div/span/div')[0])
# job_level = jobs_dom.xpath('//*[@id="contentContainer"]/div/div[2]/div/div[1]/div/div[2]/div/div[2]/div/div/div[1]/div/div/div[2]/span')[0].text
# years_experience = jobs_dom.xpath('//*[@id="contentContainer"]/div/div[2]/div/div[1]/div/div[2]/div/div[2]/div/div/div[3]/div/div/div[2]/span')[0].text
# qualifications = jobs_dom.xpath('//*[@id="contentContainer"]/div/div[2]/div/div[1]/div/div[2]/div/div[2]/div/div/div[3]/div/div/div[2]/span')[0].text
# job_type = jobs_dom.xpath('//*[@id="contentContainer"]/div/div[2]/div/div[1]/div/div[2]/div/div[2]/div/div/div[4]/div/div/div[2]/span')[0].text
# company_registration = jobs_dom.xpath('//*[@id="contentContainer"]/div/div[2]/div/div[2]/div[2]/div[2]/div/div[2]/div/div/div[1]/div/div/div[2]/span')[0].text
# company_size = jobs_dom.xpath('//*[@id="contentContainer"]/div/div[2]/div/div[2]/div[2]/div[2]/div/div[2]/div/div/div[1]/div/div/div[2]/span')[0].text
# industry_type = jobs_dom.xpath('//*[@id="contentContainer"]/div/div[2]/div/div[2]/div[2]/div[2]/div/div[2]/div/div/div[1]/div/div/div[2]/span')[0].text
# location = jobs_dom.xpath('//*[@id="contentContainer"]/div/div[1]/div[1]/div[1]/div/div/div[2]/div/div/div/div[1]/div/span')[0].text

if verbose_flag:
    print('job: ', job_title)
    print('company: ', company_name)
    print("salary range: ", salary_range)
    print('job description: ', job_description)




job:  Executive, HRIS (Developer/Data Scientist)
company:  Top Glove Corporation Berhad
salary range:  Posted on 25-Feb-22
job description:  Top Glove is embarking into a world of possibilities with a clear purpose to inspire our employees. We encourage consistent application of innovation and creativity, whether you are based in Malaysia or overseas, we find ways to break through the ordinary at the same time preserving our Corporate Value (R.I.V.E.R.), overcoming challenges and being proactive are our integral fundamentals in UNITING each and every Top Glovers. This is YOUR chance! As we are embarking into a journey to be listed in Global Fortune 500 employer in year 2030, we need more talents like you to join us.Shape YOUR future with Top Glove, The World’s Largest Manufacturer of Gloves!Your mission:1) Manage the rollout and implementation of HR modules.2) Responsible on the daily operation of HRIS.3) Provide support for HRIS including, researching and resolving HRIS problems, une

In [17]:
for job in jobs_dom.xpath('//*[@id="contentContainer"]/div/div[2]/div/div[1]/div/div[1]/div/div[2]/div/span/div/div'):
    print(etree.tostring(job))

In [18]:
cleaner = lxml.html.clean.Cleaner(allow_tags=[''])
cleaned = cleaner.clean_html(etree.tostring(jobs_dom.xpath('//*[@id="contentContainer"]/div/div[2]/div/div[1]/div/div[1]/div/div[2]/div/span/div/div')[0]))
print(cleaned.decode("utf-8").replace('<div>', '').replace('</div>',''))

IndexError: list index out of range

In [ ]:
print(etree.tostring(job_description))

b'<div class="sx2jih0 h6p8rp0 h6p8rp7 h6p8rph"><div class="sx2jih0 zcydq86q zcydq86v zcydq86w"><div class="sx2jih0 h6p8rp0 h6p8rp7 h6p8rph"><div class="sx2jih0 zcydq86q zcydq86v zcydq86w"><div class="sx2jih0 h6p8rp0 h6p8rp2"><div class="sx2jih0 zcydq866"><h4 class="sx2jih0 _18qlyvc0 _18qlyvch _1d0g9qk4 _18qlyvcs _18qlyvc1x">Job Description</h4></div><div class="sx2jih0 zcydq866"><div class="YCeva_0" data-automation="jobDescription"><span class="sx2jih0 zcydq84u _18qlyvc0 _18qlyvc1x _18qlyvc1 _18qlyvca"><div class="sx2jih0"><div><div>#BeMore</div><div style="text-align:justify"><br/>Do you want to be part of an award-winning, premium, and fast-growing outsourcing provider? Do you want to work in a company that puts your growth and happiness on top of its priorities?</div><div style="text-align:justify"><br/>We are looking for awesome additions to our growing workforce. Be home, belong, and start your #BeMore journey with TDCX today!</div><div><br/><strong>Top Reasons to work with TDCX</

In [ ]:
job_title = employer_details.find('h1', attrs={'class':'sx2jih0 _18qlyvc0 _18qlyvch _1d0g9qk4 _18qlyvcp _18qlyvc1x'}).text
print(job_title)
company = employer_details.find('span', attrs={'class':'sx2jih0 zcydq84u _18qlyvc0 _18qlyvc1x _18qlyvc2 _1d0g9qk4 _18qlyvcb'}).text
print(company)
salary_range = employer_details.findAll('span', attrs={'class':'sx2jih0 zcydq84u _18qlyvc0 _18qlyvc1x _18qlyvc1 _18qlyvca'}).
print(salary_range)

Big Data & Analytics Developer
UCSI University
[<span class="sx2jih0 zcydq84u _18qlyvc0 _18qlyvc1x _18qlyvc1 _18qlyvca">Kuala Lumpur</span>, <span class="sx2jih0 zcydq84u _18qlyvc0 _18qlyvc1x _18qlyvc1 _18qlyvca">MYR 5,000 - MYR 6,500</span>, <span class="sx2jih0 zcydq84u _18qlyvc0 _18qlyvc1x _18qlyvc1 _18qlyvca">Posted on 11-Feb-22</span>]


# Definition of scraping functions

In [105]:
def search(keyword:str, region:str) -> list:
    '''
        search jobstreet for keyword, returns results as a list of job results
    '''
    search_term = keyword.replace(' ', '-')
    root_link = r'https://www.jobstreet.com.my/en/job-search/'
    search_link = root_link + search_term + '-jobs/' + "-in-" + region
    page = requests.get(search_link)
    soup = BeautifulSoup(page.content, 'html.parser')
    results = soup.find('select', attrs={"id":"pagination"})
    options = results.find_all('option')
    num_result_pages = int(options[-1]['value'])
    print(f"searching @ {search_link}")
    print(f"total results pages for search {keyword}:", {str(num_result_pages)})

    all_results = []
    print("scraping page 1...")
    first_page_results = get_job_links(search_link)
    all_results = all_results + first_page_results

    #get the rest of the result pages
    for page in range(2, num_result_pages+1):
        print(f"scraping page {page}...")
        search_page = search_link + str(page)
        results_job_links = get_job_links(search_page)
        all_results = all_results + results_job_links

    print("done!")
    return(all_results)
    


def get_job_links(page_link:str) -> list:
    results_job_links = []
    page = requests.get(page_link)
    soup = BeautifulSoup(page.content, 'html.parser')
    jobs_list = soup.find('div', attrs={'id':"jobList"}).find('div', attrs={'data-automation':'jobListing'})
    jobs = jobslist.find_all('a', attrs={'class':'_1hr6tkx5 _1hr6tkx8 _1hr6tkxb sx2jih0 sx2jihf zcydq8h', 'href': re.compile(r'^\/en\/job\/')})
    for job in jobs:
        job_link = job.get('href')
        results_job_links.append(job_link)

    return results_job_links


def extract_job_details(job_page_link:str)->dict:
    root_link = r"https://www.jobstreet.com.my"
    full_link = root_link + job_page_link
    job_page = requests.get(full_link)
    job_soup = BeautifulSoup(job_page.content, 'html.parser')
    parsed_url = urlparse(job_fulllink)
    captured_value = parse_qs(parsed_url.query)
    
    jobID = captured_value['jobId']
    
    jobs_dom = etree.HTML(str(job_soup))
    job_title = jobs_dom.xpath('//*[@id="contentContainer"]/div/div[1]/div[1]/div[1]/div/div/div[1]/div/div/div[2]/div/div/div/div[1]/h1')[0].text
    salary_range = jobs_dom.xpath('//*[@id="contentContainer"]/div/div[1]/div[1]/div[1]/div/div/div[2]/div/div/div/div[2]/span')[0].text

    cleaner = lxml.html.clean.Cleaner(allow_tags=[''])
    cleaned = cleaner.clean_html(etree.tostring(jobs_dom.xpath('//*[@id="contentContainer"]/div/div[2]/div/div[1]/div')[0]))
                                                                #//*[@id="contentContainer"]/div/div[2]/div/div[1]/div/div[1]/div/div[2]/div/span/div
    job_description = cleaned.decode("utf-8").replace('<div>', '').replace('</div>','')
    job_level = list(list(job_soup.find('span', string="Career Level").parent.parent.children)[1].children)[0].text                        
    experience = list(list(job_soup.find('span', string="Years of Experience").parent.parent.children)[1].children)[0].text                       
    job_type = list(list(job_soup.find('span', string="Job Type").parent.parent.children)[1].children)[0].text 
    qualifications = list(list(job_soup.find('span', string="Qualification").parent.parent.children)[1].children)[0].text
    job_specialization = list(list(job_soup.find('span', string="Job Specializations").parent.parent.children)[1].children)[0].text


    company_name = jobs_dom.xpath('//*[@id="contentContainer"]/div/div[1]/div[1]/div[1]/div/div/div[1]/div/div/div[2]/div/div/div/div[2]/span')[0].text
    company_registration = list(list(job_soup.find('span', string="Registration No.").parent.parent.children)[1].children)[0].text
    company_size = list(list(job_soup.find('span', string="Company Size").parent.parent.children)[1].children)[0].text
    company_industry = list(list(list(job_soup.find('span', string="Industry").parent.parent.children)[1].children)[0].children)[0].text
    
    return {"job_id":jobID, 'job_title': job_title, "salary_range":salary_range, "job_description":job_description, "job_level":job_level, "experience":experience, "job_type":job_type, "qualifications":qualifications, 'job_specialization':job_specialization, 'company_name':company_name, 'company_registration':company_registration, "company_size":company_size, "company_industry":company_industry, }
    

def get_job_page(job_page_link:str):
    root_link = r"https://www.jobstreet.com.my"
    full_link = root_link + job_page_link
    job_page = requests.get(full_link)
    job_soup = BeautifulSoup(job_page.content, 'html.parser')
    parsed_url = urlparse(job_fulllink)
    captured_value = parse_qs(parsed_url.query)
    
    job_ID = captured_value['jobId']

    return {'job_ID': job_ID, 'page':page.content}
    


In [41]:
states = ["negeri-sembilan", "kedah", "pahang", "perak", "selangor", "sabah", "johor", "sarawak", "terengganu", "perlis", "kelantan", "melaka", "penang", "kuala-lumpur"]
job_search_terms = ["data engineer", "data scientist", "data analyst"]


results = search("data engineer")

searching @ https://www.jobstreet.com.my/en/job-search/data-engineer-jobs/
total results pages for search data engineer: {'30'}
scraping page 1...
scraping page 2...
scraping page 3...
scraping page 4...
scraping page 5...
scraping page 6...
scraping page 7...
scraping page 8...
scraping page 9...
scraping page 10...
scraping page 11...
scraping page 12...
scraping page 13...
scraping page 14...
scraping page 15...
scraping page 16...
scraping page 17...
scraping page 18...
scraping page 19...
scraping page 20...
scraping page 21...
scraping page 22...
scraping page 23...
scraping page 24...
scraping page 25...
scraping page 26...
scraping page 27...
scraping page 28...
scraping page 29...
scraping page 30...
done!


In [42]:
print(len(results))
print(results[1])

900
/en/job/assistant-vice-president-data-engineer-compliance-amlt-remediation-4824511?jobId=jobstreet-my-job-4824511&sectionRank=2&token=0~2cdf76e9-a859-4c15-bb9a-e16a948b507a&fr=SRP%20Job%20Listing


In [106]:
JD = extract_job_details(results[1])
JD2 = extract_job_details(results[3])


In [107]:
df = pd.DataFrame(JD)

In [108]:
df = df.append(JD2, ignore_index=True)

In [109]:
df.head()

,job_id,job_title,salary_range,job_description,job_level,experience,job_type,qualifications,job_specialization,company_name,company_registration,company_size,company_industry
0,jobstreet-my-job-4846899,Assistant Vice President - Data Engineer ( Com...,Posted on 25-Feb-22,"Job Description• Design, build and implement E...",Manager,5 years,Full-Time,"Bachelor's Degree, Post Graduate Diploma, Prof...","Accounting/Finance, Banking/Financial",CIMB Group,13491-P,More than 5000 Employees,Banking / Financial Services
1,[jobstreet-my-job-4846899],Data Engineer,Posted on 25-Feb-22,Job HighlightsYou’re guaranteed a Flexible Wor...,Senior Executive,4 years,Full-Time,Not Specified,"Computer/Information Technology, IT-Software",KINESSO MALAYSIA SDN. BHD. (fka Mediabrands Gl...,201201019216,2001 - 5000 Employees,Advertising / Marketing / Promotion / PR


In [73]:
page = requests.get(r"https://www.jobstreet.com.my/en/job/assistant-vice-president-data-engineer-compliance-amlt-remediation-4824511?fr=SRP%20Job%20Listing&jobId=jobstreet-my-job-4824511&sectionRank=2&token=0~2cdf76e9-a859-4c15-bb9a-e16a948b507a")

In [87]:
soup = BeautifulSoup(page.content, 'html.parser')
list(list(soup.find('span', string="Career Level").parent.parent.children)[1].children)[0].text

'Manager'